<a href="https://colab.research.google.com/github/rohitgarg2411/CNN-architectures-implemetation-in-pytorch/blob/master/DenseNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import numpy as np
import torchvision.transforms as transforms
import torchvision
import torch.nn as nn
import torch.nn.functional as F

In [0]:
class bottle_neck(nn.Module):
  def __init__(self,in_channels,k):
    #k=growth rate
    super(bottle_neck,self).__init__()
    self.bn1=nn.BatchNorm2d(in_channels)
    self.conv1=nn.Conv2d(in_channels,4*k,1)
    self.bn2=nn.BatchNorm2d(4*k)
    self.conv2=nn.Conv2d(4*k,k,3,padding=1)

  def forward(self,x):
    output=F.relu(self.bn1(x))
    output=F.relu(self.bn2(self.conv1(output)))
    output=self.conv2(output)
    output=torch.cat([x,output],1)
    return output








In [0]:
class transition(nn.Module):
  def __init__(self,in_channels,out_channels):
    super(transition,self).__init__()
    self.bn=nn.BatchNorm2d(in_channels)
    self.conv1=nn.Conv2d(in_channels,out_channels,1)
    self.avgpool=nn.AvgPool2d(2,stride=2)
  
  def forward(self,x):
    output=self.conv1(self.bn(x))
    output=self.avgpool(output)
    return output

In [0]:
 class DenseNet(nn.Module):

   def __init__(self,block,nblocks,growth_rate=12,reduction=0.5,num_class=100):
     super(DenseNet,self).__init__()
     self.growth_rate=growth_rate
     inner_channels=2*growth_rate
     self.conv1=nn.Conv2d(3,inner_channels,3,padding=1)
     self.bn1=nn.BatchNorm2d(inner_channels)
     self.features=nn.Sequential()

     for i in range(len(nblocks)-1):
       self.features.add_module("dense_block_layer{}".format(i),self.make_dense_layers(block,inner_channels,nblocks[i]))
       inner_channels+=growth_rate*nblocks[i]
       out_channels=int(reduction*inner_channels)
       self.features.add_module("transition_layer_{}".format(i), transition(inner_channels, out_channels))
       inner_channels=out_channels

     self.features.add_module("dense_block{}".format(len(nblocks) - 1), self.make_dense_layers(block, inner_channels, nblocks[len(nblocks)-1]))
     inner_channels += growth_rate * nblocks[len(nblocks) - 1]
     self.features.add_module('bn', nn.BatchNorm2d(inner_channels))
     self.features.add_module('relu', nn.ReLU())
     self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
     self.linear = nn.Linear(inner_channels, num_class)

   def forward(self, x):
        output = self.bn1(self.conv1(x))
        output = self.features(output)
        output = self.avgpool(output)
        #print(output.size()[0])
        output = output.view(output.size()[0], -1)
        output = self.linear(output)
        return output

   def make_dense_layers(self, block, in_channels, nblocks):
        dense_block = nn.Sequential()
        for index in range(nblocks):
            dense_block.add_module('bottle_neck_layer_{}'.format(index), block(in_channels, self.growth_rate))
            in_channels += self.growth_rate
        return dense_block 


     
              

      

In [0]:
def group_label(label):

    if label<=9 or 35<=label<=44 or 55<=label<=69 or 75<=label<=84:
       return 0
    elif 10<=label<=14 or 20<=label<=24 or 84<=label<=89:
       return 1
    elif 15<=label<=19 or 25<=label<=34 or label>=90:
       return 2 
    elif 45<=label<=54:
       return 3
    elif 70<=label<=74:
       return 4

  


In [0]:
loaded_model=DenseNet(bottle_neck,[16,16,16])
loaded_model.load_state_dict(torch.load("/content/drive/My Drive/rohit/model1.pth"))
loaded_model.eval()

In [0]:

transform1 = transforms.Compose(
        [transforms.RandomCrop(32,padding=4),
         transforms.RandomHorizontalFlip(),
         transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
        )
transform2 = transforms.Compose(
        [transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
        )

    #Load train and test set:

train = torchvision.datasets.CIFAR100(root='/content/drive/My Drive/datasets',train=True,download=True,transform=transform1)
trainset = torch.utils.data.DataLoader(train,batch_size=64,shuffle=True)

test = torchvision.datasets.CIFAR100(root='/content/drive/My Drive/datasets',train=False,download=True,transform=transform2)
testset = torch.utils.data.DataLoader(test,batch_size=64,shuffle=False)
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

net = loaded_model
net.to(device)
costFunc = torch.nn.CrossEntropyLoss()
optimizer =  torch.optim.SGD(net.parameters(),lr=0.01,momentum=0.9,nesterov=True,weight_decay=0.0001)
scheduler=torch.optim.lr_scheduler.StepLR(optimizer,step_size=25,gamma=0.1)

'''for epoch in range(50):
        closs = 0
        total=0
        correctHits=0
        for i,batch in enumerate(trainset):
            data,output = batch
            data,output = data.to(device),output.to(device)
            prediction = net(data)
            loss = costFunc(prediction,output)
            closs = loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            #print every 100th time
            if i%100 == 0:
                print('[%d  %d] loss: %f'% (epoch+1,i+1,closs))
            _,prediction = torch.max(prediction.data,1)  #returns max as well as its index
            total += output.size(0)
            correctHits += (prediction==output).sum().item()
        accu=100.0*correctHits/total
        for param_group in optimizer.param_groups:
            print("learning rate= "+str(param_group['lr']))
        print(f'training_accuracy ={accu}')'''
        
    #test
with torch.no_grad():
         correctHits=0
         total=0
         n_class_correct = [0 for i in range(100)]
         n_class_samples = [0 for i in range(100)]
         for batch_no,batches in enumerate(testset):
            data,output = batches
            data,output = data.to(device),output.to(device)
            prediction = net(data)
            _,prediction = torch.max(prediction.data,1)  #returns max as well as its index
            total += output.size(0)
            correctHits += (prediction==output).sum().item()

            for i in range(64 if batch_no!=156 else 16):
                  label = output[i]
                  pred = prediction[i]
                  if (label == pred):
                     n_class_correct[label] += 1
                  n_class_samples[label] += 1
          
         accu=100.0*correctHits/total
         print(f'testing_accuracy ={accu}')
         scheduler.step()


n_group_correct = [0 for i in range(5)]
n_group_samples = [0 for i in range(5)]
for i in range(100):
            grp=group_label(i)
            n_group_correct[grp]+=n_class_correct[i]
            n_group_samples[grp]+=n_class_samples[i]

for i in range(5):      
           print(n_group_samples[i])
           acc=100.0*n_group_correct[i]/n_group_samples[i]
           print(f'accuracy for group {i+1} is {acc} %')    


Files already downloaded and verified
Files already downloaded and verified
cuda
testing_accuracy =72.86
4500
accuracy for group 1 is 73.84444444444445 %
1500
accuracy for group 2 is 76.4 %
2500
accuracy for group 3 is 70.88 %
1000
accuracy for group 4 is 72.7 %
500
accuracy for group 5 is 63.6 %


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


In [0]:
torch.save(loaded_model.state_dict(),"/content/drive/My Drive/rohit/model1.pth")